In [1]:
import serial
import datetime
import serial.tools.list_ports
import pandas as pd

In [2]:
studentRecord = pd.read_csv("studentRecords.csv")
visitHistory = dict({"studentID":[], "name":[],"major":[],"Sign-in Time":[],"Sign-out Time":[]})
roomStatus = dict({"Students In":[]})
historyDF = pd.DataFrame()
studentRecord.head()

,studentID,name,major,Sign-in Time,Sign-out Time,signingIn
0,801606786,Samuel VanDenburgh,MECE,0,0,True
1,801735307,Noah Clippinger,Computer Engineering,0,0,True


In [3]:
comlist = serial.tools.list_ports.comports() #finds barcode port and opens it
connected = []
for element in comlist:
    connected.append(element.device)
    if element.description == "barcode scanner":
        ser = serial.Serial(element.device)
    print("Barcode Scanner connected at:", ser.name)

Barcode Scanner connected at: /dev/ttyACM1


In [4]:
def storeVisit(student): #Keeps a list of all recorded scans
        visitHistory["studentID"].append(student["studentID"])
        visitHistory["name"].append(student["name"])
        visitHistory["major"].append(student["major"])
        visitHistory["Sign-in Time"].append(student["Sign-in Time"])
        visitHistory["Sign-out Time"].append(student["Sign-out Time"]) #store values into JSON

In [5]:
def signingIn(student): #Keeps a list of all recorded scans
        #edit students scan status and time
        student["Sign-in Time"] = datetime.datetime.now().time()
        student['signingIn'] = "False"
        print("Student Signed In!")

In [6]:
def signingOut(student): #Keeps a list of all recorded scans
        #edit students scan status and histoyr
        student["Sign-out Time"] = datetime.datetime.now().time()
        #record visit
        student['signingIn'] = "True"
        print("Student Signed Out :(")

In [7]:
def getStudent(line):
    #find student with matching ID
    for student in studentRecord:
        if student["studentID"] == line:
            return student
        else:
            print("Student Not Found")

In [8]:
while True: #checks for new input from barcode and updates dataframe
    line = ser.readline()
    print("line: ", line)
    line = line.decode('utf-8',errors='ignore').rstrip('/r/n')
    line = int(line)
    student = studentRecord.loc[studentRecord['studentID'] == line]
    ser.flush()
    print("\n\nName",student.name,"\n\n")
    if str(student['signingIn']) == "True": #If student is signing in
        signingIn(student)
    else: #If student is signing out
        signingOut(student)
        storeVisit(student)
    #If time is pass hours
    #    for student in Students

line:  b'801606786\r\n'


Name 0    Samuel VanDenburgh
Name: name, dtype: object 


Student Signed Out :(


/tmp/ipykernel_59086/3876710155.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student["Sign-out Time"] = datetime.datetime.now().time()
/tmp/ipykernel_59086/3876710155.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student['signingIn'] = "True"


SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)